In [42]:
import numpy as np
import pandas as pd
from scipy import stats
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

### Задание 3

Смоделируйте 100 выборок по 30 наблюдений в каждой из нормального распределения
N(0; 1).

In [34]:
n = 30
samples = []
for i in range(100):
    samples.append(np.random.normal(0, 1, n))

а) Считая дисперсию неизвестной, проверьте для каждой выборки гипотезу о равенстве нулю среднего с уровнем значимости 0.05. Сколько гипотез было отвергнуто?

Воспользуемся одновыборочным t-критерием Стьюдента.

Ответ: для 9 из 100 выборок H0 отклонилась.

In [35]:
ntrue = 0
nfalse = 0
alpha = 0.05
interval = stats.t.interval(1-alpha, df=n-1)
print(f"Confidence interval: {interval}, alpha={alpha}")
for sample in samples:
    T = np.sqrt(n) * np.mean(sample) / np.std(sample)
    if T < interval[0] or T > interval[1]:
        print(f"H0 rejected with T={T}")
        nfalse += 1
    else:
        ntrue += 1
        
print("----")
print(f"Rejected: {nfalse}, not rejected: {ntrue}")

Confidence interval: (-2.045229642132703, 2.045229642132703), alpha=0.05
H0 rejected with T=2.2265394601370763
H0 rejected with T=2.19037872801441
H0 rejected with T=2.317520892371165
H0 rejected with T=2.2575313762108844
H0 rejected with T=4.000261473658974
H0 rejected with T=-2.0926245134332353
H0 rejected with T=2.1557134058866545
H0 rejected with T=2.28083412996442
H0 rejected with T=-2.068485810932098
----
Rejected: 9, not rejected: 91


б) Сколько гипотез будет отвергнуто, если вы сделаете поправку на множественные сравнения (по Бонферрони и по Зидаку)?

Бонферрони (отклонилась 1 гипотеза)

In [36]:
alpha_b = alpha / 100
ntrue = 0
nfalse = 0
interval = stats.t.interval(1-alpha_b, df=n-1)
print(f"Confidence interval: {interval}, alpha={alpha_b}")
for sample in samples:
    T = np.sqrt(n) * np.mean(sample) / np.std(sample)
    if T < interval[0] or T > interval[1]:
        print(f"H0 rejected with T={T}")
        nfalse += 1
    else:
        ntrue += 1
        
print("----")
print(f"Rejected: {nfalse}, not rejected: {ntrue}")

Confidence interval: (-3.9177141183790445, 3.9177141183790445), alpha=0.0005
H0 rejected with T=4.000261473658974
----
Rejected: 1, not rejected: 99


Зидак (отклонилась 1 гипотеза)

In [37]:
alpha_z = 1 - (1 - alpha)**(1/100)
ntrue = 0
nfalse = 0
interval = stats.t.interval(1-alpha_z, df=n-1)
print(f"Confidence interval: {interval}, alpha={alpha_z}")
for sample in samples:
    T = np.sqrt(n) * np.mean(sample) / np.std(sample)
    if T < interval[0] or T > interval[1]:
        print(f"H0 rejected with T={T}")
        nfalse += 1
    else:
        ntrue += 1
        
print("----")
print(f"Rejected: {nfalse}, not rejected: {ntrue}")

Confidence interval: (-3.9083626146214816, 3.9083626146214816), alpha=0.0005128014162623096
H0 rejected with T=4.000261473658974
----
Rejected: 1, not rejected: 99


в) Проверьте для всех пар выборок гипотезы о равенстве средних (используйте t-тест с уровнем значимости 0.05). Если бы вы сделаете поправку по Бонферрони, то сколько гипотез отвергнется?

Ответ: таким образом, без поправки отвергается 350/4950 (7%) гипотез, с ней - ни одной.

In [38]:
total_comparisons = int(100*99/2)
alpha = 0.05
alpha_b = alpha/total_comparisons
rejected_simple = 0
rejected_bonferroni = 0
for i in range (0, 100):
    for j in range(i+1, 100):
        sample1 = samples[i]
        sample2 = samples[j]
        statistics, pvalue = stats.ttest_ind(sample1, sample2)
        if pvalue < alpha:
            rejected_simple += 1
        if pvalue < alpha_b:
            rejected_bonferroni += 1
print(f"[Simple] rejected: {rejected_simple}/{total_comparisons}")
print(f"[Bonferroni] rejected: {rejected_bonferroni}/{total_comparisons}")

[Simple] rejected: 350/4950
[Bonferroni] rejected: 0/4950


г) Воспользуйтесь тестом Тьюки, чтобы проверить эти гипотезы. Сколько теперь гипотез будет отвергнуто?

In [39]:
mean_v = sorted([np.mean(sample) for sample in samples], reverse=True)
s = np.std(np.concatenate(samples))
critical_value = 4.47 # from
rejected_tukey = 0
for i in range(0, 100):
    for j in range(i+1, 100):
        T = (mean_v[i] - mean_v[j]) / (s * np.sqrt(2/n))
        if T > critical_value:
            rejected_tukey += 1
print(f"[Tukey] rejected: {rejected_tukey}/{total_comparisons}")

[Tukey] rejected: 0/4950


### Задание 4

Смоделируйте выборку объема 400 из распределения Exp(7) (обозначим эту выборку через X = {x1, . . . , x400}.
Для i = 1, . . ., 398 гипотеза H(i)0 состоит в том, что подвыборка {xi, xi+1, . . . , x400} имеет распределение Exp(6). Воспользуйтесь алгоритмом Хольма, чтобы проверить эти гипотезы. Сколько гипотез было бы отвергнуто, если бы
вы проверяли все гипотезы независимо с поправкой по Бонферрони?

In [115]:
def check_bonferroni(sample, m, alpha=0.05):
    p_value = stats.kstest(rvs=sample, cdf='expon', args=(0, 1/6))[1]
    return False if (p_value < alpha/m) else True
 
def holm(samples, alpha=0.05):
    m = len(samples)
    p_values = []
    for sample in samples:
        p_values.append(stats.kstest(rvs=sample, cdf='expon', args=(0, 1/6))[1])
    ids = np.arange(len(samples))
    tuples = sorted([(p_value, sid) for (p_value, sid) in zip(p_values, ids)])
    rejected_ids = []
    for i, (p_value, sid) in enumerate(tuples):
        if p_value < alpha/(m - i):
            rejected_ids.append(sid)
        else:
            break
    return rejected_ids

In [130]:
sample = np.random.exponential(scale=1/7, size=400)

In [131]:
subsamples = [sample[i:] for i in range(0, 398)]

rejected_holm_ids = holm(subsamples)
rejected_bonferroni_ids = []
for sid, subsample in enumerate(subsamples):
    if not check_bonferroni(subsample, 398):
        rejected_bonferroni_ids.append(sid)

In [103]:
print(f"Holm rejected: {len(rejected_holm_ids)}")
print(f"Bonferroni rejected: {len(rejected_bonferroni_ids)}")

Holm rejected: 96
Bonferroni rejected: 85


Таким образом, с помощью алгоритма Хольма отклонилось больше неверных гипотез, чем с помощью Бонферрони.

In [2]:
df = pd.read_csv('./IB_statistics_data_HW9.txt', sep=' ')
df.head(3)

,Ex1,Ex1_group,Ex2,Ex2_group,Ex5,Ex5_group
0,1.782074,1,1.626210,1,-4.020372,1
1,13.437385,1,-0.330518,1,1.929893,1
2,6.821891,1,-8.453625,1,1.765300,1


### Задание 1
(1 балл) Вам даны две выборки из нормального распределения. Проверьте гипотезу
о том, что у них одинаковое среднее, сначала предполагая выборки независимыми, а
затем предполагая их зависимыми. Найдите p-значение в обоих случаях.

In [9]:
sample1 = df[df.Ex1_group == 1].Ex1.values
sample2 = df[df.Ex1_group == 2].Ex1.values

Независимые выборки (гипотеза подтвердилась)

In [18]:
n1 = sample1.shape[0]
n2 = sample1.shape[0]
mean1 = np.mean(sample1)
mean2 = np.mean(sample2)
std1sq = np.std(sample1)**2
std2sq = np.std(sample2)**2

t = (mean1-mean2)/np.sqrt(std1sq/n1 + std2sq/n2)
frdeg = (std1sq/n1 + std2sq/n2)**2 / ((std1sq/n1)**2/(n1-1) + (std2sq/n2)**2/(n2-1))
critical_interval = stats.t.interval(0.95, df=frdeg)
print(f"Critical interval: {critical_interval}")
print(f"Statistics: {t}")
print(f"P-value: {stats.t.cdf(t, df=frdeg)}")
if t < critical_interval[0] or t > critical_interval[1]:
    print(f"H0 is rejected!")
else:
    print(f"H0 is not rejected!")

Critical interval: (-1.9717573833265514, 1.9717573833265514)
Statistics: -0.3320616456529388
P-value: 0.3700932152648305
H0 is not rejected!


Зависимые выборки (гипотеза подтвердилась)

In [20]:
n = sample1.shape[0]
y_diff = sample1 - sample2
dmean = np.mean(y_diff)
dstd = np.std(y_diff)
t = np.sqrt(n) * dmean / dstd
frdeg = n-1
critical_interval = stats.t.interval(0.95, df=frdeg)
print(f"Critical interval: {critical_interval}")
print(f"Statistics: {t}")
print(f"P-value: {stats.t.cdf(t, df=frdeg)}")
if t < critical_interval[0] or t > critical_interval[1]:
    print(f"H0 is rejected!")
else:
    print(f"H0 is not rejected!")

Critical interval: (-1.98326414470971, 1.98326414470971)
Statistics: -0.3431859159084037
P-value: 0.3660790336065417
H0 is not rejected!


### Задание 2
(1 балл) Вам дана выборка из нормального распределения, разбитая на 4 подгруппы.
Воспользуйтесь критерием Фишера, чтобы проверить гипотезу о равенстве средних в
подгруппах.

Ответ: гипотеза подтвердилась

In [28]:
sample1 = df[df.Ex2_group == 1].Ex2.values
sample2 = df[df.Ex2_group == 2].Ex2.values
sample3 = df[df.Ex2_group == 3].Ex2.values
sample4 = df[df.Ex2_group == 4].Ex2.values

r = 4
n = df.Ex2.values.shape[0]

y_mean = np.mean(df.Ex2.values)
y1_mean = np.mean(sample1)
y2_mean = np.mean(sample2)
y3_mean = np.mean(sample3)
y4_mean = np.mean(sample4)

Q1 = (y1_mean-y_mean)**2 + (y2_mean-y_mean)**2 + (y3_mean-y_mean)**2 + (y4_mean-y_mean)**2
Q2 = np.sum((sample1 - y1_mean)**2) + np.sum((sample2 - y2_mean)**2) + \
     np.sum((sample3 - y3_mean)**2) + np.sum((sample4 - y4_mean)**2)

F = (n-r)*Q1/((r-1)*Q2)
critical_value = stats.f.ppf(0.95, r-1, n-r)
print(f"Critical value: {critical_value}")
print(f"Statistics: {F}")
print(f"P-value: {1 - stats.f.cdf(F, r-1, n-r)}")
if t > critical_value:
    print(f"H0 is rejected!")
else:
    print(f"H0 is not rejected!")

Critical value: 2.6488634148319594
Statistics: 0.008380648857972923
P-value: 0.9989439430898426
H0 is not rejected!


### Задание 3

In [134]:
n = df.shape[0]
n1 = df[df.Ex2_group == 1].shape[0]
n2 = df[df.Ex2_group == 2].shape[0]
n3 = df[df.Ex2_group == 3].shape[0]
n4 = df[df.Ex2_group == 4].shape[0]

X = np.zeros([n, 5])
X[:, 0] = 1
X[0:n1, 1] = 1
X[n1:n1+n2, 2] = 1
X[n1+n2:n1+n2+n3, 3] = 1
X[n1+n2+n3:, 4] = 1

y = df.Ex2.values

In [173]:
XTX = X.T.dot(X)
XTY = X.T.dot(y)
A = np.zeros((XTX.shape[0] + 1, XTX.shape[1]))
A[:XTX.shape[0], :] = XTX
A[-1, 1:] = 1
z = np.zeros(XTY.shape[0] + 1)
z[:-1] = XTY
beta = np.linalg.lstsq(A, z, rcond=None)[0]
y_pred = np.dot(X, beta)
errors = y - y_pred
sigma_squared = np.dot(errors.T, errors)/(X.shape[0] - X.shape[1])
covM_inv = np.linalg.inv(sigma_squared*XTX_inv[1:, 1:])
statistics = beta[1:].dot(covM_inv).dot(beta[1:])
critical_value = stats.chi2.ppf(0.05, df=4)
print(f"Estimation of beta: {beta}")
print(f"Estimation of sigma-squared: {sigma_squared}")
print(f"Statistics: {statistics}, critical value: {critical_value}")
if statistics > critical_value:
    print("H0 is rejected!")
else:
    print("H0 is not rejected!")

Estimation of beta: [ 0.27478646 -0.01448844  0.10734959 -0.52317517  0.43031402]
Estimation of sigma-squared: 19.32801846943315
Statistics: 0.967380995281384, critical value: 0.7107230213973245
H0 is rejected!


### Задание 4
Найти строго (1, 3, 4)-обратную матрицу

O1: AA−A = A

O2: A−AA− = A−

O3: (AA−)∗ = AA−

O4:  (A−A)∗ = A−A

Решение.

Пользуясь определениями свойств O1-O4 получаем систему уравнений (если будет необходимо, вышлю черновик, где это выводится):

$$2a_{11} + a_{12} + a_{13} = 1$$
$$3a_{12} + 1 + a_{13} = 0$$
$$4a_{13} + 1 = 0$$
$$a_{21} = a_{11} - 1$$
$$a_{22} = a_{12} + 1$$
$$a_{23} = a_{13}$$
$$a_{23} = a_{13}$$
$$a_{31} = a_{11} - 1$$
$$a_{32} = a_{12}$$
$$a_{32} = a_{12}$$
$$a_{33} = 1 + a_{13}$$

Решением системы уравнений является матрица, приведенная ниже. Для нее все свойства выполняются, значит построение строго (1,3,4)-обратной матрицы для А с вещественными числами невозможно.

In [75]:
def is_O1(A, A_):
    n = A.flatten().shape[0]
    return np.sum(A.dot(A_).dot(A) == A) == n

def is_O2(A, A_):
    n = A_.flatten().shape[0]
    return np.sum(A_.dot(A).dot(A_) == A_) == n

def is_O3(A, A_):
    m1 = A.dot(A_).T
    m2 = A.dot(A_)
    n = m1.flatten().shape[0]
    return np.sum(m1 == m2) == n

def is_O4(A, A_):
    m1 = A_.dot(A).T
    m2 = A_.dot(A)
    n = m1.flatten().shape[0]
    return np.sum(m1 == m2) == n

In [156]:
A = np.array([[1, 0, 0, 1],
              [0, 1, 0, 1],
              [0, 0, 1, 1]])
print("A")
print(A)

A_ = np.array([[3/4, -1/4, -1/4],
               [-1/4, 3/4, -1/4],
               [-1/4, -1/4, 3/4],
               [1/4, 1/4, 1/4]])

print("A-")
print(A_)

A
[[1 0 0 1]
 [0 1 0 1]
 [0 0 1 1]]
A-
[[ 0.75 -0.25 -0.25]
 [-0.25  0.75 -0.25]
 [-0.25 -0.25  0.75]
 [ 0.25  0.25  0.25]]


In [117]:
print(f"Property O1: {is_O1(A, A_)}")
print(f"Property O2: {is_O2(A, A_)}")
print(f"Property O3: {is_O3(A, A_)}")
print(f"Property O4: {is_O3(A, A_)}")

Property O1: True
Property O2: True
Property O3: True
Property O4: True


### Задание 5

Находясь в модели однофакторного дисперсионного анализа, постройте МНК оценки
неизвестных параметров, используя обобщенно обратные матрицы.

In [182]:
n = df.shape[0]
n1 = df[df.Ex5_group == 1].shape[0]
n2 = df[df.Ex5_group == 2].shape[0]
n3 = df[df.Ex5_group == 3].shape[0]
n4 = df[df.Ex5_group == 4].shape[0]

X = np.zeros([n, 5])
X[:, 0] = 1
X[0:n1, 1] = 1
X[n1:n1+n2, 2] = 1
X[n1+n2:n1+n2+n3, 3] = 1
X[n1+n2+n3:, 4] = 1

y = df.Ex5.values

5.1 Здесь можем использовать просто псевдообратную матрицу, которая дает решение с минимальной нормой.

In [215]:
XTX_inv = np.linalg.pinv(np.dot(X.T, X))
beta = np.dot(np.dot(XTX_inv, X.T), y)
y_pred = np.dot(X, beta)
errors = y - y_pred
sigma_squared = np.dot(errors.T, errors)/(X.shape[0] - X.shape[1])
print(f"Estimation of beta: {beta}")
print(f"Estimation of sigma-squared: {sigma_squared}")

Estimation of beta: [-0.07482351 -0.22887273  0.02160736  0.05789431  0.07454755]
Estimation of sigma-squared: 4.05672822406411


5.2 Берем псевдообратную матрицу из пункта 1, которая дает решение с минимальной нормой и также является матрицей A(1,3).
Подбираем на ее основе такую матрицу A(1,3), которая минимизирует дополнительное условие.

In [202]:
def Z_gen(m, cnt):
    for i in range(0, cnt):
        yield np.random.randf([m, m])

def limitation1(n_v, beta_v):
    return np.sum(1/n_v*np.abs(beta_v))

In [218]:
A_plus = XTX_inv
A = np.dot(X.T, X)
I = np.zeros([5, 5])
n_v = np.array([n1, n2, n3, n4])
m = 5
np.fill_diagonal(I, 1)

XTX_inv = np.linalg.pinv(np.dot(X.T, X))
beta = np.dot(np.dot(XTX_inv, X.T), y)

best_beta = beta
best_limitation = limitation1(n_v, beta[1:])
print(f"Best limitation before: {best_limitation}")   
print(f"Best beta before {best_beta}")

for Z in Z_gen(5, 100000):
    new_A_plus = A_plus + (I - A_plus.dot(A)).dot(Z)
    new_beta = np.dot(np.dot(new_A_plus, X.T), y)
    limit_val = limitation1(n_v, new_beta[1:])
    if limit_val < best_limitation:
        best_limitation = limit_val
        best_beta = new_beta

print(f"Best limitation after: {best_limitation}")   
print(f"Best beta after {best_beta}")

Best limitation before: 0.007772261859872714
Best beta before [-0.07482351 -0.22887273  0.02160736  0.05789431  0.07454755]
Best limitation after: 0.006643654061699556
Best beta after [-0.03274636 -0.27094988 -0.02046979  0.01581716  0.0324704 ]


Таким образом

In [221]:
y_pred = np.dot(X, best_beta)
errors = y - y_pred
sigma_squared = np.dot(errors.T, errors)/(X.shape[0] - X.shape[1])
print(f"Estimation of beta: {best_beta}")
print(f"Estimation of sigma-squared: {sigma_squared}")

Estimation of beta: [-0.03274636 -0.27094988 -0.02046979  0.01581716  0.0324704 ]
Estimation of sigma-squared: 4.05672822406411


5.3 Делаем тоже самое, что и в 5.2, но с другим условием

In [222]:
def limitation2(beta):
    return np.max(np.abs(beta))

In [224]:
A_plus = XTX_inv
A = np.dot(X.T, X)
I = np.zeros([5, 5])
n_v = np.array([n1, n2, n3, n4])
m = 5
np.fill_diagonal(I, 1)

XTX_inv = np.linalg.pinv(np.dot(X.T, X))
beta = np.dot(np.dot(XTX_inv, X.T), y)

best_beta = beta
best_limitation = limitation2(beta[1:])
print(f"Best limitation before: {best_limitation}")   
print(f"Best beta before {best_beta}")

for Z in Z_gen(5, 100000):
    new_A_plus = A_plus + (I - A_plus.dot(A)).dot(Z)
    new_beta = np.dot(np.dot(new_A_plus, X.T), y)
    limit_val = limitation2(new_beta[1:])
    if limit_val < best_limitation:
        best_limitation = limit_val
        best_beta = new_beta

print(f"Best limitation after: {best_limitation}")   
print(f"Best beta after {best_beta}")

y_pred = np.dot(X, best_beta)
errors = y - y_pred
sigma_squared = np.dot(errors.T, errors)/(X.shape[0] - X.shape[1])
print()
print(f"Estimation of beta: {best_beta}")
print(f"Estimation of sigma-squared: {sigma_squared}")

Best limitation before: 0.2288727316251592
Best beta before [-0.07482351 -0.22887273  0.02160736  0.05789431  0.07454755]
Best limitation after: 0.20383011193868317
Best beta after [-0.09986613 -0.20383011  0.04664998  0.08293693  0.09959017]

Estimation of beta: [-0.09986613 -0.20383011  0.04664998  0.08293693  0.09959017]
Estimation of sigma-squared: 4.05672822406411
